In [1]:
import pandas as pd

In [2]:
df=pd.read_csv('train.csv', sep=',', encoding='utf8')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42000 entries, 0 to 41999
Columns: 785 entries, label to pixel783
dtypes: int64(785)
memory usage: 251.5 MB


In [5]:
y = df['label']
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = \
train_test_split(df, y, test_size=0.33, random_state=1)

In [13]:
# Удаляем таргет
X_train = X_train.drop(['label'], axis=1)
X_test = X_test.drop(['label'], axis=1)

In [14]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [15]:
# Инициализируем модели:

# КНН
model = Pipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier(n_neighbors=7))
])

In [16]:
#обучение КНН
model.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('knn', KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=7, p=2,
           weights='uniform'))])

In [17]:
y_knn = model.predict_proba(X_test, )

In [20]:
df_test = pd.read_csv('test.csv', sep=',', encoding='utf8')

In [21]:
#Делаем предсказание для выборки test
X_predict = model.predict(df_test)

In [23]:
#Запишем массив с предсказание в датафрейм
df_predict = pd.DataFrame(X_predict)

In [37]:
df_predict.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Data columns (total 1 columns):
0    28000 non-null int64
dtypes: int64(1)
memory usage: 218.8 KB


In [24]:
# Вызываем из датафрейма столбец типа Series
df_pr=df_predict[0]

In [25]:
import numpy as np

In [39]:
df_numbers = np.arange(1, len(X_predict)+1) #Сделаем первый столбец с нумерацией по порядку начиная с 1 для ImageId

In [40]:
df_num_series = pd.Series(df_numbers) #переводим numpy array в series

In [41]:
df_num = pd.DataFrame(df_num_series)

In [43]:
df_num.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28000 entries, 0 to 27999
Data columns (total 1 columns):
0    28000 non-null int32
dtypes: int32(1)
memory usage: 109.5 KB


In [44]:
# Объединяем таргет и ID в датафрейм
df_result=pd.concat([df_num, df_pr], axis=1)

In [45]:
# Правильно называем колонки
df_result.columns = ['ImageId', 'Label']

In [46]:
# Записываем в файл
df_result.to_csv('test_predictions.csv',index=False)

In [ ]:
#ЛОГИЧЕСКАЯ РЕГРЕССИЯ

In [50]:
from sklearn.linear_model import LinearRegression, LogisticRegression

In [51]:
#Логическая регрессия
logreg = LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=500, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=1e-10,
          verbose=0, warm_start=False)

In [52]:
#обучение логической регрессии
logreg.fit(X_train, y_train)

LogisticRegression(C=100, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=500, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=1e-10,
          verbose=0, warm_start=False)

In [53]:
y_logreg = logreg.predict_proba(X_test, )

In [54]:
#Делаем предсказание для выборки test
X_predict_logreg = logreg.predict(df_test)

In [55]:
#Запишем массив с предсказание в датафрейм
df_predict_logreg = pd.DataFrame(X_predict_logreg)

In [56]:
df_pr_logreg=df_predict_logreg[0]

In [57]:
# Объединяем таргет и ID в датафрейм
df_result_logreg=pd.concat([df_num, df_pr_logreg], axis=1)

In [58]:
# Правильно называем колонки
df_result_logreg.columns = ['ImageId', 'Label']

In [59]:
# Записываем в файл
df_result_logreg.to_csv('test_predictions_logreg.csv',index=False)

In [ ]:
# Метод дерева принятия решений

In [61]:
from sklearn.tree import DecisionTreeClassifier

In [62]:
# Дерево принятия решений
tree = DecisionTreeClassifier(max_depth=4, max_features=44)

In [63]:
#обучение дерева принятия решений
tree.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=44, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [ ]:
#Расчёт лучших параметров для дерева принятия решений
from sklearn.model_selection import GridSearchCV, cross_val_score
tree_params = {'max_depth': range(1,50),
'max_features': range(4,50)}
tree_grid = GridSearchCV(tree, tree_params,cv=30, n_jobs=-1,verbose=True)
tree_grid.fit(X_train, y_train)
tree_grid.best_params_, tree_grid.best_score_

Fitting 30 folds for each of 2254 candidates, totalling 67620 fits


[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:   15.5s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:  1.3min
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  5.3min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed: 12.1min
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed: 16.6min
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed: 21.7min
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed: 27.3min
[Parallel(n_jobs=-1)]: Done 4984 tasks      | elapsed: 33.9min
[Parallel(n_jobs=-1)]: Done 6034 tasks      | elapsed: 41.1min
[Parallel(n_jobs=-1)]: Done 7184 tasks      | elapsed: 49.0min
[Parallel(n_jobs=-1)]: Done 8434 tasks      | elapsed: 56.6min
[Parallel(n_jobs=-1)]: Done 9784 tasks      | elapsed: 64.9min
[Parallel(n_jobs=-1)]: Done 11234 tasks      | elapsed: 74.3min
[Parallel(n_jobs=-1)]: Done 12784 tasks      | elapsed: 85

In [ ]:
y_tree = tree.predict_proba(X_test, )